In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

importing the disaster half of the data

In [3]:
df = pd.read_csv('data_with_coordinates.csv')

we analyze what we have so far

In [4]:
df.head()
df.info()
df.describe()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61702 entries, 0 to 61701
Data columns (total 49 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   DisNo.                                     61702 non-null  object 
 1   Historic                                   61702 non-null  object 
 2   Classification Key                         61702 non-null  object 
 3   Disaster Group                             61702 non-null  object 
 4   Disaster Subgroup                          61702 non-null  object 
 5   Disaster Type                              61702 non-null  object 
 6   Disaster Subtype                           61702 non-null  object 
 7   External IDs                               18009 non-null  object 
 8   Event Name                                 15685 non-null  object 
 9   ISO                                        61702 non-null  object 
 10  Country               

Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No. Injured', 'No. Affected', 'No. Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update', 'location_query',
       'Latitude_cached', 'Longitude_cached', 'Latitude', 'Longitude'],
      dtype='object')

Now we do some data cleaning before trying to merge the data

In [5]:
df = df.dropna(subset=['Latitude', 'Longitude'])
columns_to_drop = [
    'External IDs', 'OFDA/BHA Response', 'Appeal', 'Declaration',
    'Insured Damage (\'000 US$)', 'Insured Damage, Adjusted (\'000 US$)',
    'Reconstruction Costs (\'000 US$)', 'Reconstruction Costs, Adjusted (\'000 US$)',
    'Total Damage (\'000 US$)', 'Total Damage, Adjusted (\'000 US$)',
    'Admin Units', 'Entry Date', 'Last Update', 'DisNo.', 'Historic', 'Magnitude Scale', 'Origin', 'Associated Types', 'River Basin', 'CPI', 'Magnitude', 'Classification Key', 'Total Deaths', 'Total Affected','No. Injured', 'No. Affected', 'No. Homeless', 'Latitude_cached', 'location_query', 'Longitude_cached', 'Disaster Group', 'Disaster Subgroup', 'Event Name', 'ISO', 'AID Contribution (\'000 US$)'
]

df = df.drop(columns=columns_to_drop, errors='ignore')  # ignore if any column is missing
df.head()
df.to_csv('cleaned_coordinate_data.csv', index=False)
df.columns

Index(['Disaster Type', 'Disaster Subtype', 'Country', 'Subregion', 'Region',
       'Location', 'Start Year', 'Start Month', 'Start Day', 'End Year',
       'End Month', 'End Day', 'Latitude', 'Longitude'],
      dtype='object')

In [6]:
print(df['Disaster Subtype'].value_counts())

Disaster Subtype
Riverine flood                      8111
Tropical cyclone                    5384
Flood (General)                     4805
Drought                             2530
Flash flood                         2262
Bacterial disease                   1552
Cold wave                           1539
Viral disease                       1386
Storm (General)                     1267
Blizzard/Winter storm               1241
Road                                1097
Tornado                             1008
Heat wave                            973
Severe weather                       897
Lightning/Thunderstorms              849
Extra-tropical storm                 656
Ground movement                      578
Severe winter conditions             544
Forest fire                          518
Landslide (wet)                      504
Water                                479
Fire (Miscellaneous)                 361
Explosion (Industrial)               356
Air                                  228

In [7]:
# Normalize the 'Disaster Subtype' column (convert to lowercase, strip spaces)
df['Disaster Subtype'] = df['Disaster Subtype'].astype(str).str.strip().str.lower()

# Define allowed subtypes (also normalized to lowercase and stripped)
valid_subtypes = [s.lower().strip() for s in [
    # Floods
    'Riverine flood',
    'Flood (General)',
    'Flash flood',
    'Coastal flood',

    # Storms
    'Tropical cyclone',
    'Storm (General)',
    'Lightning/Thunderstorms',
    'Hail',
    'Severe weather',

    # Temperature Events
    'Heat wave',
    'Cold wave',

    # Drought
    'Drought',

    # Optional Fire-related
    'Forest fire',
    'Wildfire (General)',
    'Land fire (Brush, Bush, Pasture)'
]]

# Drop all rows not in the valid_subtypes list — IN PLACE
df.drop(df[~df['Disaster Subtype'].isin(valid_subtypes)].index, inplace=True)

# Optional: reset index
df.reset_index(drop=True, inplace=True)
df


,Disaster Type,Disaster Subtype,Country,Subregion,Region,Location,Start Year,Start Month,Start Day,End Year,End Month,End Day,Latitude,Longitude
0,Drought,drought,Djibouti,Sub-Saharan Africa,Africa,Ali Sabieh,2001,6.0,NaN,2001,NaN,NaN,11.163069,42.837278
1,Drought,drought,Djibouti,Sub-Saharan Africa,Africa,Dikhil,2001,6.0,NaN,2001,NaN,NaN,11.428370,42.063977
2,Drought,drought,Djibouti,Sub-Saharan Africa,Africa,Djibouti,2001,6.0,NaN,2001,NaN,NaN,11.814597,42.845306
3,Drought,drought,Djibouti,Sub-Saharan Africa,Africa,Obock,2001,6.0,NaN,2001,NaN,NaN,11.964015,43.292228
4,Drought,drought,Sudan,Northern Africa,Africa,Northern Darfur,2000,1.0,NaN,2001,NaN,NaN,9.203465,26.916471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29866,Drought,drought,Kenya,Sub-Saharan Africa,Africa,Mandera,2025,1.0,NaN,2025,NaN,NaN,3.228533,40.705615
29867,Drought,drought,Kenya,Sub-Saharan Africa,Africa,Marsabit,2025,1.0,NaN,2025,NaN,NaN,2.857958,37.715489
29868,Drought,drought,Kenya,Sub-Saharan Africa,Africa,Turkana,2025,1.0,NaN,2025,NaN,NaN,3.525895,36.074295
29869,Drought,drought,Kenya,Sub-Saharan Africa,Africa,Samburu,2025,1.0,NaN,2025,NaN,NaN,1.539446,36.942166


kept data for africa only for simplicity....if we get to get more data i'll add the other surrounding areas 

In [8]:
# Normalize 'Region' column: lowercase and strip
df['Region'] = df['Region'].astype(str).str.strip().str.lower()

# Drop rows where Region is not 'africa'
df.drop(df[df['Region'] != 'africa'].index, inplace=True)

# Optional: reset index
df.reset_index(drop=True, inplace=True)
df.head()

,Disaster Type,Disaster Subtype,Country,Subregion,Region,Location,Start Year,Start Month,Start Day,End Year,End Month,End Day,Latitude,Longitude
0,Drought,drought,Djibouti,Sub-Saharan Africa,africa,Ali Sabieh,2001,6.0,NaN,2001,NaN,NaN,11.163069,42.837278
1,Drought,drought,Djibouti,Sub-Saharan Africa,africa,Dikhil,2001,6.0,NaN,2001,NaN,NaN,11.428370,42.063977
2,Drought,drought,Djibouti,Sub-Saharan Africa,africa,Djibouti,2001,6.0,NaN,2001,NaN,NaN,11.814597,42.845306
3,Drought,drought,Djibouti,Sub-Saharan Africa,africa,Obock,2001,6.0,NaN,2001,NaN,NaN,11.964015,43.292228
4,Drought,drought,Sudan,Northern Africa,africa,Northern Darfur,2000,1.0,NaN,2001,NaN,NaN,9.203465,26.916471


Filling Missing Values in the dates columns

In [12]:
import pandas as pd
import numpy as np

# Forward-fill start date fields (no warnings now)
df[['Start Year', 'Start Month', 'Start Day']] = df[['Start Year', 'Start Month', 'Start Day']].ffill()

# Forward-fill end date fields
df[['End Year', 'End Month', 'End Day']] = df[['End Year', 'End Month', 'End Day']].ffill()

# Ensure types are numeric
date_cols = ['Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day']
df[date_cols] = df[date_cols].apply(pd.to_numeric, errors='coerce')

# Adjust end dates to be >= start dates
for i in df.index:
    if pd.notnull(df.at[i, 'End Year']) and df.at[i, 'End Year'] < df.at[i, 'Start Year']:
        df.at[i, 'End Year'] = df.at[i, 'Start Year']

    if df.at[i, 'End Year'] == df.at[i, 'Start Year']:
        if pd.notnull(df.at[i, 'End Month']) and df.at[i, 'End Month'] < df.at[i, 'Start Month']:
            df.at[i, 'End Month'] = df.at[i, 'Start Month']

    if (df.at[i, 'End Year'] == df.at[i, 'Start Year'] and
        df.at[i, 'End Month'] == df.at[i, 'Start Month']):
        if pd.notnull(df.at[i, 'End Day']) and df.at[i, 'End Day'] < df.at[i, 'Start Day']:
            df.at[i, 'End Day'] = df.at[i, 'Start Day']

df.dropna(subset=['Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day'], inplace=True)
df.reset_index(drop=True, inplace=True)


df.head()

,Disaster Type,Disaster Subtype,Country,Subregion,Region,Location,Start Year,Start Month,Start Day,End Year,End Month,End Day,Latitude,Longitude
0,Flood,riverine flood,Angola,Sub-Saharan Africa,africa,Benguela province),2000,1.0,8.0,2000,1.0,15.0,-12.910466,14.035661
1,Flood,riverine flood,Angola,Sub-Saharan Africa,africa,Kuanza Norte province),2000,1.0,8.0,2000,1.0,15.0,-9.029675,15.092632
2,Flood,riverine flood,Mozambique,Sub-Saharan Africa,africa,Matutuine,2000,1.0,26.0,2000,3.0,27.0,-26.459698,32.574586
3,Flood,riverine flood,Mozambique,Sub-Saharan Africa,africa,Manhica,2000,1.0,26.0,2000,3.0,27.0,-25.288938,32.882790
4,Flood,riverine flood,Mozambique,Sub-Saharan Africa,africa,Magude,2000,1.0,26.0,2000,3.0,27.0,-24.759805,32.438285


We try to enrich the data

In [ ]:
import pandas as pd
import requests
from datetime import datetime

API_HOST = "meteostat.p.rapidapi.com"
API_KEY = "YOUR_RAPIDAPI_KEY"

headers = {
    "x-rapidapi-host": API_HOST,
    "x-rapidapi-key": API_KEY
}

def build_date(year, month, day):
    return f"{int(year):04d}-{int(month):02d}-{int(day):02d}"

def find_nearest_station(lat, lon):
    url = f"https://meteostat.p.rapidapi.com/stations/nearby?lat={lat}&lon={lon}"
    response = requests.get(url, headers=headers)
    stations = response.json().get('data', [])
    return stations[0]['id'] if stations else None

def fetch_weather_data(station_id, start_date, end_date):
    url = "https://meteostat.p.rapidapi.com/stations/daily"
    params = {
        "station": station_id,
        "start": start_date,
        "end": end_date,
        "units": "metric"
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json().get('data', [])

def aggregate_weather(data):
    if not data:
        return {}
    df = pd.DataFrame(data)
    return {
        "avg_temp": df['tavg'].mean(),
        "max_temp": df['tmax'].max(),
        "min_temp": df['tmin'].min(),
        "total_precip": df['prcp'].sum(),
        "avg_wind_speed": df['wspd'].mean(),
        "sunshine_total": df['tsun'].sum()
    }

# Load your CSV
df = pd.read_csv("your_disaster_data.csv")

# Enrich each row
enriched_rows = []
for idx, row in df.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    start = build_date(row["Start Year"], row["Start Month"], row["Start Day"])
    end = build_date(row["End Year"], row["End Month"], row["End Day"])
    
    station_id = find_nearest_station(lat, lon)
    if not station_id:
        print(f"No station found for {lat},{lon}")
        continue
    
    weather_data = fetch_weather_data(station_id, start, end)
    weather_summary = aggregate_weather(weather_data)
    
    enriched_row = row.to_dict()
    enriched_row.update(weather_summary)
    enriched_row["disaster_occurred"] = 1
    enriched_rows.append(enriched_row)

# Save enriched dataset
enriched_df = pd.DataFrame(enriched_rows)
enriched_df.to_csv("disaster_enriched.csv", index=False)
